In [2]:
#!pip install chardet
#!pip install lxml
import chardet
import lxml.etree as ET
import pandas as pd

# Exemplo 1 - XML dos "escâneres" -  equipamentos de inspeção não invasiva

## Problema de encoding???!!!

In [3]:
with open('ex1.xml') as txt:
    print(txt.read())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

## Use chardet !!!

In [4]:
with open('ex1.xml', 'rb') as raw:
    raw_xml = raw.read()
    encode = chardet.detect(raw_xml)
    print(encode)
    txt_xml = raw_xml.decode(encode['encoding'])
    print(txt_xml)

{'encoding': 'UTF-16', 'confidence': 1.0, 'language': ''}
<DataForm><Version>V01.07</Version><TruckId>20190701002631000A</TruckId><State>5</State><Site>002631</Site><Date>2019-07-01T11:15:30.117Z</Date><InTraining>0</InTraining><InReference>0</InReference><Archived>0</Archived><InEdition>0</InEdition><Analysed>1</Analysed><CheckedOut>0</CheckedOut><Approved>0</Approved><Pending>0</Pending><AdminData><FileId/><Comments/><Vehicle><PlateNumber>ESU6715</PlateNumber><OCR>-1</OCR><Trailers><Trailer><PlateNumber/><OCR>-1</OCR><Containers><Container><ContainerId>EITU0158854</ContainerId><OCR>-1</OCR></Container><Container><ContainerId/><OCR>-1</OCR></Container></Containers></Trailer></Trailers><Driver/></Vehicle></AdminData><ProcessInstructions><InspectionType Value="0"/><EnergyLevel Value="0"/><PendingRequired>0</PendingRequired><ScanPosition>0</ScanPosition><HEDRequired>0</HEDRequired><Speed>24</Speed><IsContainerEmpty>0</IsContainerEmpty></ProcessInstructions><Operations><Operation Type="1"

## "Marretada" para ler vários tipos de tag...

In [5]:


TAGS_NUMERO = ['ContainerId', 'container_no', 'ContainerID1', 'Container1']
TAGS_DATA = ['Date', 'SCANTIME', 'ScanTime', 'createdate']

numeros = []
datas = []
try:
    root = ET.fromstring(txt_xml)
    for atag in TAGS_NUMERO:
        for tag in root.iter(atag):
            numeros.append(tag.text)
    for atag in TAGS_DATA:
        for tag in root.iter(atag):
            datas.append(tag.text)
    print(numeros, datas)
except ET.ParseError as err:
    print(err)


['EITU0158854', None] ['2019-07-01T11:15:30.117Z']


Este código é uma simplificacão didática de um código utilizado no Sistema Ajna

No sistema Ajna, é preciso ler XMLs produzidos por 18 equipamentos de escaneamento de contêineres diferentes. São campos contendo informação do número de contêiner, da data de escaneamento, do operador de escâner, se houve alerta, entre outros.

Além de encodings diversos, cada equipamento coloca nomes ligeiramente diferentes nas tags. Alguns ainda possuem namespaces XML ou ao invés de utilizar tags utilizam atributos. O código inteiro pode ser conferido em:

https://github.com/IvanBrasilico/virasana/blob/master/virasana/integracao/xmli.py

Exemplos de arquivos XML podem ser consultados na API do Ajna

# Exemplo 2 - XMLs do Mercante

In [6]:
with open('ex2.XML') as txt:
    print(txt.read())

<?xml version='1.0' encoding='ISO-8859-1' standalone='yes'?><cargas versaoXml='2'>
<conhecimentosEmbarque>
<tipoMovimento><![CDATA[A]]></tipoMovimento>
<dataAtualizacao><![CDATA[2019-12-14]]></dataAtualizacao>
<horaAtualizacao><![CDATA[20:47:51]]></horaAtualizacao>
<numeroCEmercante><![CDATA[071905281576276]]></numeroCEmercante>
<tipoTrafego><![CDATA[5]]></tipoTrafego>
<numConhecimento><![CDATA[MEDUN4773895]]></numConhecimento>
<tipoBLConhecimento><![CDATA[11]]></tipoBLConhecimento>
<quantidadeFilhotes><![CDATA[1]]></quantidadeFilhotes>
<cnpjAgenteDesconsolidador><![CDATA[02735565000908]]></cnpjAgenteDesconsolidador>
<empresaNVOCC><![CDATA[CN001982]]></empresaNVOCC>
<dataEmissao><![CDATA[2019-10-28]]></dataEmissao>
<cubagem>0000000049.510</cubagem>
<portoDestFinal><![CDATA[BRSUA]]></portoDestFinal>
<portoOrigemCarga><![CDATA[CNNGB]]></portoOrigemCarga>
<descricao><![CDATA[10 PK REDE DE PLASTICO DE 400CM DE LARGURA DENOMINADO MEIO DE INFUSAO FABRICADO EM PLASTICO POLIAMIDA UTILIZADO EM 

## Exemplo simples fazendo "na mão"

In [7]:
xtree = ET.parse('ex2.XML')
xroot = xtree.getroot()
objetos = []

for node in xroot:
    if node.tag == 'conhecimentosEmbarque':
        ce = node.find('numeroCEmercante').text
        bl = node.find('numConhecimento').text
        objetos.append({'CE': ce, 'BL': bl})
df = pd.DataFrame(objetos)


In [8]:
print(objetos)

[{'CE': '071905281576276', 'BL': 'MEDUN4773895'}, {'CE': '071905282302781', 'BL': 'NGB81263867'}, {'CE': '071905281581601', 'BL': 'MEDUSM240454'}, {'CE': '071905282302862', 'BL': 'SHA81276750'}]


In [9]:
df.head()

,CE,BL
0,071905281576276,MEDUN4773895
1,071905282302781,NGB81263867
2,071905281581601,MEDUSM240454
3,071905282302862,SHA81276750


## Exemplo simples com função e parâmetros

In [22]:
def simple_XMLparser(xroot, texto_classe:str, campos: list)-> pd.DataFrame:
    objetos = []
    nodes_classe = xroot.findall(texto_classe)
    for node_classe in nodes_classe:
        linha = {}
        for campo in campos:
            try:
                node_campo = node_classe.find(campo)
                if node_campo is not None:
                    linha[campo] = node_campo.text
            except Exception as err:
                print(err)
        objetos.append(linha)
    return pd.DataFrame(objetos)


In [35]:
# Apenas para visualização do retorno de find e findall:
nodes_classe = xroot.findall('conhecimentosEmbarque')
print(nodes_classe)
um_node_campo = nodes_classe[0].find('portoDestFinal')
print(um_node_campo, um_node_campo.tag, um_node_campo.text)
none_node_campo = nodes_classe[0].find('nonecsiste')
print(none_node_campo)
print(none_node_campo.text)  #  Erro!!!

[<Element conhecimentosEmbarque at 0x7f748c2bdd88>, <Element conhecimentosEmbarque at 0x7f748c2bd908>, <Element conhecimentosEmbarque at 0x7f748c2bdd48>, <Element conhecimentosEmbarque at 0x7f748c2bdf88>]
<Element portoDestFinal at 0x7f748c316ec8> portoDestFinal BRSUA
None


AttributeError: 'NoneType' object has no attribute 'text'

In [30]:
df = simple_XMLparser(xroot, 'conhecimentosEmbarque', 
                       [
                        'numeroCEmercante',
                        'numConhecimento',
                        'dataEmissao'
                       ])
     

In [31]:
df.head()

,numeroCEmercante,numConhecimento,dataEmissao
0,071905281576276,MEDUN4773895,2019-10-28
1,071905282302781,NGB81263867,2019-10-28
2,071905281581601,MEDUSM240454,2019-11-07
3,071905282302862,SHA81276750,2019-11-07


## Exemplo completo utilizando classes


Neste exemplo, além de mapear mais campos, são mapeados também estruturas "filhas" do conhecimento no XML, através de property e classes "filhas"

In [9]:
from xml.etree.ElementTree import Element
class XMLMapper:

    def _campos(self):
        return [campo for campo in dir(self) if campo[0] != '_']

    def _parse_node(self, node):
        for campo in self._campos():
            # print(campo)
            alvo = node.find(campo)
            if alvo is not None:
                # print(alvo.text, alvo.tag)
                try:
                    destino = getattr(self, campo)
                except AttributeError as err:
                    logger.error(err)
                    continue
                if isinstance(destino, str):
                    setattr(self, campo, alvo.text)
                else:
                    setattr(self, campo, alvo)
            else:
                pass
                # logger.info('Não encontrou %s' % campo)

    def _to_dict(self):
        result = {}
        for campo in self._campos():
            valor = getattr(self, campo)
            if isinstance(valor, XMLMapper):
                valor = str(valor)
            result[campo] = valor
        return result


    
class Consignatario(XMLMapper):
    def __init__(self):
        self.tipoConsignatario: str = ''
        self.cnpjConsignatario: str = ''
        self.nomeConsignatarioEstrangeiro: str = ''
        self.dadosComplementaresConsignatario: str = ''

    def __repr__(self):
        if (self.cnpjConsignatario is not None and
                self.cnpjConsignatario and
                self.cnpjConsignatario != ''):
            return self.cnpjConsignatario
        return self.nomeConsignatarioEstrangeiro


class Embarcador(XMLMapper):
    def __init__(self):
        self.cnpjShipper: str = ''
        self.idEmbarcador: str = ''

    def __repr__(self):
        if self.cnpjShipper:
            return self.cnpjShipper
        return self.idEmbarcador

class ManifestoCE(XMLMapper):
    def __init__(self):
        self.numeroManifesto: str = ''

    def __repr__(self):
        return self.numeroManifesto

    
class Conhecimento(XMLMapper):
    def __init__(self):
        self.tipoMovimento: str = ''
        self.dataAtualizacao: str = ''
        self.horaAtualizacao: str = ''
        self.tipoTrafego: str = ''
        self.tipoBLConhecimento: str = ''
        self.numeroCEMaster: str = ''
        self.dataEmissao: str = ''
        self.cubagem: str = ''
        self.portoDestFinal: str = ''
        self.portoOrigemCarga: str = ''
        self.descricao: str = ''
        self.numeroCEmercante: str = ''
        self._embarcador: Embarcador = None
        self._consignatario: Consignatario = None
        self.codigoTerminalCarregamento: str = ''
        self.paisDestinoFinalMercante: str = ''
        self.indicadorShipsConvenience: str = ''
        self._manifestoCE: ManifestoCE = None
        self.codigoEmpresaNavegacao: str = ''

    @property
    def embarcador(self) -> str:
        return self._embarcador

    @embarcador.setter
    def embarcador(self, node: Element):
        self._embarcador = Embarcador()
        self._embarcador._parse_node(node)

    @property
    def consignatario(self) -> str:
        return self._consignatario

    @property
    def dadoscomplementaresconsignatario(self) -> str:
        if self._consignatario:
            return self._consignatario.dadosComplementaresConsignatario
        return ''

    @consignatario.setter
    def consignatario(self, node: Element):
        self._consignatario = Consignatario()
        self._consignatario._parse_node(node)

    @property
    def manifestoCE(self) -> str:
        return self._manifestoCE

    @manifestoCE.setter
    def manifestoCE(self, node: Element):
        self._manifestoCE = ManifestoCE()
        self._manifestoCE._parse_node(node)


In [10]:
xroot = xtree.getroot()
objetos = []
for node in xroot:
    objeto = Conhecimento()
    objeto._parse_node(node)
    objetos.append(objeto._to_dict())
df = pd.DataFrame(objetos)
df.reset_index()
df.head()
# Dica: o Pandas, com um comando apenas, pode salvar o resultado em uma tabela SQL!!!
# engine = sqlalchemy.create_engine('sqlite:///teste.db')
# df.to_sql(node.tag, engine, if_exists='append', index=False)


,codigoEmpresaNavegacao,codigoTerminalCarregamento,consignatario,cubagem,dadoscomplementaresconsignatario,dataAtualizacao,dataEmissao,descricao,embarcador,horaAtualizacao,indicadorShipsConvenience,manifestoCE,numeroCEMaster,numeroCEmercante,paisDestinoFinalMercante,portoDestFinal,portoOrigemCarga,tipoBLConhecimento,tipoMovimento,tipoTrafego
0,02378779001504,,02735565000908,0000000049.510,DSV UTI AIR & SEA AGENCIAMENTO DE TRANSPORTES ...,2019-12-14,2019-10-28,10 PK REDE DE PLASTICO DE 400CM DE LARGURA DEN...,"DSV AIR & SEA CO., LTD. NINGBO BRANCH AS AGENT...",20:47:51,,0719502495273,,071905281576276,,BRSUA,CNNGB,11,A,5
1,,,None,,,,,,None,,,None,,,,,,,,
2,02735565000908,,17596025000100,0000000049.510,LM WIND POWER DO BRASIL SA,2019-12-14,2019-10-28,01 X 40 STC 10 PACKAGE(s) REDE DE PLASTICO DE ...,"SHANGHAI LEADGOTECH CO,LTD",20:47:51,,0719502495273,071905281576276,071905282302781,,BRSUA,CNNGB,12,I,5
3,,,None,,,,,,None,,,None,,,,,,,,
4,02378779001504,,02735565000908,0000000003.000,DSV UTI AIR & SEA AGENCIAMENTO DE TRANSPORTES ...,2019-12-14,2019-11-07,2 PE FIBERGLASS MAT NCM:7019 NET WEIGHT:1248KG...,"DSV AIR & SEA CO., LTD",20:47:57,,0719502495273,,071905281581601,,BRSUA,CNSHA,11,A,5


In [11]:
print(objetos)

[{'codigoEmpresaNavegacao': '02378779001504', 'codigoTerminalCarregamento': '', 'consignatario': '02735565000908', 'cubagem': '0000000049.510', 'dadoscomplementaresconsignatario': 'DSV UTI AIR & SEA AGENCIAMENTO DE TRANSPORTES LTDA', 'dataAtualizacao': '2019-12-14', 'dataEmissao': '2019-10-28', 'descricao': '10 PK REDE DE PLASTICO DE 400CM DE LARGURA DENOMINADO MEIO DE INFUSAO FABRICADO EM PLASTICO POLIAMIDA UTILIZADO EM PROCESSO DE INFUSAO DE FABRICACAO DE PAS EOLICAS TECIDO FABRICADO EM NAILON SEM FIOS DE BORRACHA COM PESO EM DECITEX DE 225 COM MAIS DE DOIS FILAMENTOS NCM CODE:3926/5407 WOODEN PACKAGE: PROCESSED', 'embarcador': 'DSV AIR & SEA CO., LTD. NINGBO BRANCH AS AGENT FOR DSV OCEAN TRANSPORT A/S', 'horaAtualizacao': '20:47:51', 'indicadorShipsConvenience': '', 'manifestoCE': '0719502495273', 'numeroCEMaster': '', 'numeroCEmercante': '071905281576276', 'paisDestinoFinalMercante': '', 'portoDestFinal': 'BRSUA', 'portoOrigemCarga': 'CNNGB', 'tipoBLConhecimento': '11', 'tipoMovime

Exemplo completo do ETL dos XMLs do Mercante:

https://github.com/IvanBrasilico/virasana/tree/master/virasana/integracao/mercante